# Fine-tune Qwen3 for Dino Assistant

In [ ]:
!pip install -q bitsandbytes>=0.46.1 transformers accelerate peft datasets

In [ ]:
import json, os
!git clone https://github.com/RekitRex21/Dino-Training.git
os.chdir('Dino-Training')

In [ ]:
data = []
for f in ['security_analysis.json', 'trading.json']:
    with open(f) as fp: data.extend(json.load(fp))
with open('rex_assistant.jsonl') as fp:
    for line in fp: data.append(json.loads(line))
print(f"Loaded {len(data)} examples")

In [ ]:
formatted_data = [{'text': f"Instruction: {i['instruction']}\n\nInput: {i.get('input','')}\n\nOutput: {i['output']}"} for i in data]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct", trust_remote_code=True)

In [ ]:
from datasets import Dataset
ds = Dataset.from_list(formatted_data)
ds = ds.map(lambda x: tokenizer(x['text'], truncation=True, max_length=2048, padding='max_length'), batched=True)
print(f"Tokenized: {len(ds)} examples")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type='nf4', bnb_4bit_compute_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-7B-Instruct", quantization_config=bnb, device_map='auto', trust_remote_code=True)
print("Model loaded!")

In [ ]:
from peft import LoraConfig, get_peft_model
lora = LoraConfig(r=8, lora_alpha=16, target_modules=["q_proj","v_proj","k_proj","o_proj"], lora_dropout=0.05, bias='none', task_type='CAUSAL_LM')
model = get_peft_model(model, lora)
model.print_trainable_parameters()

In [ ]:
from transformers import Trainer, TrainingArguments
args = TrainingArguments(output_dir='./qwen3-dino', per_device_train_batch_size=2, gradient_accumulation_steps=4, learning_rate=2e-4, num_train_epochs=3, logging_steps=10)
trainer = Trainer(model=model, args=args, train_dataset=ds)
trainer.train()

In [ ]:
model.save_pretrained('./qwen3-dino-final')
tokenizer.save_pretrained('./qwen3-dino-final')
print('Saved!')

In [ ]:
# Merge LoRA and convert to GGUF for Ollama
merged = model.merge_and_unload()
merged.save_pretrained('./qwen3-dino-merged')
!git clone -q https://github.com/ggerganov/llama.cpp
!python llama.cpp/convert.py ./qwen3-dino-merged --outfile qwen3-dino.gguf --outtype q4_0
print('GGUF ready!')

In [ ]:
# Create Modelfile
modelfile = '''FROM ./qwen3-dino.gguf
PARAMETER temperature 0.7
SYSTEM You are Rex's Dino assistant. You know about DinoScan security scanner, Dino Dynasty OS, trading bots, OpenClaw skills.
'''
with open('Modelfile', 'w') as f: f.write(modelfile)
print('Modelfile created!')

In [ ]:
# DOWNLOAD: Right-click qwen3-dino.gguf and Modelfile in Files panel!
print('Download qwen3-dino.gguf and Modelfile from Files panel!')
print('Then on your PC: ollama create rex-dino-qwen3 -f Modelfile')